In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification
!unzip /content/gtzan-dataset-music-genre-classification.zip # gtzan rozpakuj

In [ ]:
!cp /content/drive/MyDrive/Music_Dataset/dataset.zip . 
!cp /content/drive/MyDrive/Music_Dataset/CNN_CLASIFIER_annotations_withgtzan_v2.csv .
!unzip dataset.zip

In [ ]:
import os
# USUN DISCO I COUNTRY PRZERZUC DO FOLDERU
for dir in os.listdir("/content/Data/genres_original"):
  for file in os.listdir("/content/Data/genres_original/"+dir):
    if "disco" in file:
      os.remove("/content/Data/genres_original/"+dir+"/"+file)
    elif "country" in file:
      os.remove("/content/Data/genres_original/"+dir+"/"+file)
    else:
      if file.endswith(".wav"):
        os.rename("/content/Data/genres_original/"+dir+"/"+file, "/content/dataset/"+file)


In [ ]:
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio

class AudioUtil():
  # ----------------------------
  # Load an audio file. Return the signal as a tensor and the sample rate
  # ----------------------------
  @staticmethod
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    return (sig, sr)

  # ----------------------------
  # Convert the given audio to the desired number of channels
  # ----------------------------
  @staticmethod
  def rechannel(aud, new_channel):
    sig, sr = aud

    if (sig.shape[0] == new_channel):
      # Nothing to do
      return aud

    if (new_channel == 1):
      # Convert from stereo to mono by selecting only the first channel
      resig = sig[:1, :]
    elif (new_channel == 3):
      resig = sig[:1, :]
      resig = torch.cat(3*[resig])
      '''
      print(sig.shape)
      print(sig.size())
      channel = sig[0,:]
      
      resig = sig.expand(3, -1)

      x[0,:] = channel
      x[1,:] = channel
      x[2,:] = channel
      '''
    else:
      # Convert from mono to stereo by duplicating the first channel
      resig = torch.cat([sig, sig])

    return ((resig, sr))

  # ----------------------------
  # Since Resample applies to a single channel, we resample one channel at a time
  # ----------------------------
  @staticmethod
  def resample(aud, newsr):
    sig, sr = aud

    if (sr == newsr):
      # Nothing to do
      return aud

    num_channels = sig.shape[0]
    # Resample first channel
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      # Resample the second channel and merge both channels
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))

  # ----------------------------
  # Pad (or truncate) the signal to a fixed length 'max_ms' in milliseconds
  # ----------------------------
  @staticmethod
  def pad_trunc(aud, max_ms): # TU WCHODZI ILE MA TRWAC
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms 

    if (sig_len > max_len):
      # Truncate the signal to the given length
      sig = sig[:,:max_len]

    elif (sig_len < max_len):
      # Length of padding to add at the beginning and end of the signal
      pad_begin_len = random.randint(0, max_len - sig_len)
      pad_end_len = max_len - sig_len - pad_begin_len

      # Pad with 0s
      pad_begin = torch.zeros((num_rows, pad_begin_len))
      pad_end = torch.zeros((num_rows, pad_end_len))

      sig = torch.cat((pad_begin, sig, pad_end), 1)
      
    return (sig, sr)

  # ---------------------------- DATA AUGUMENTATION
  # Shifts the signal to the left or right by some percent. Values at the end
  # are 'wrapped around' to the start of the transformed signal.
  # ----------------------------
  @staticmethod
  def time_shift(aud, shift_limit):
    sig,sr = aud
    _, sig_len = sig.shape
    shift_amt = int(random.random() * shift_limit * sig_len)
    return (sig.roll(shift_amt), sr)
  
  # ----------------------------
  # Generate a Spectrogram
  # ----------------------------
  @staticmethod
  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80

    # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

    # Convert to decibels
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)
  

  # ---------------------------- MEL AUGUMENTATION
  # Augment the Spectrogram by masking out some sections of it in both the frequency
  # dimension (ie. horizontal bars) and the time dimension (vertical bars) to prevent
  # overfitting and to help the model generalise better. The masked sections are
  # replaced with the mean value.
  # ----------------------------
  @staticmethod
  def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec

    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
      aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
      aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

    return aug_spec

from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio

# ----------------------------
# Sound Dataset
# ----------------------------
class SoundDS(Dataset):
  def __init__(self, df, data_path, duration=30000, shift_pct=0.4):
    self.df = df #
    self.data_path = str(data_path) # /content/dataset/
    self.duration = duration # tu bylo 4000 ms czyli 4s
    self.sr = 44100
    self.channel = 3 # ZMIANA NA 3 kanaly zeby sie zgadzalo
    self.shift_pct = shift_pct
            
  # ----------------------------
  # Number of items in dataset
  # ----------------------------
  def __len__(self):
    return len(self.df)    
    
  # ----------------------------
  # Get i'th item in dataset
  # ----------------------------
  def __getitem__(self, idx):
    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path
    audio_file = self.data_path + self.df.loc[idx, 'filename'] # tu bylo relative_path zamiast filename
    # Get the Class ID
    class_id = self.df.loc[idx, 'label'] # tu bylo classID zamiast label
    class_id = int(class_id)
    aud = AudioUtil.open(audio_file) # jakby dodac to.device!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # Some sounds have a higher sample rate, or fewer channels compared to the
    # majority. So make all sounds have the same number of channels and same 
    # sample rate. Unless the sample rate is the same, the pad_trunc will still
    # result in arrays of different lengths, even though the sound duration is
    # the same.
    reaud = AudioUtil.resample(aud, self.sr)
    rechan = AudioUtil.rechannel(reaud, self.channel)

    dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
    shift_aud = AudioUtil.time_shift(dur_aud, self.shift_pct)
    sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None) # tu mozna sie pobawic parametrami
    aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

    return aug_sgram, class_id

# save tensor to png
nie dziala
**Save spectrograms to png for faster training**



```
DATASET ZWRACA 

return aug_sgram, class_id, self.df.loc[idx, 'filename']
```



In [ ]:
import pandas as pd
import numpy as np


BATCH_SIZE = 1

df = pd.read_csv("/content/CNN_CLASIFIER_annotations_withgtzan_v2.csv")
data_path = "/content/dataset/"

myds = SoundDS(df, data_path)
mel_png = torch.utils.data.DataLoader(myds, batch_size=BATCH_SIZE, shuffle=False)

print(myds.size())

In [ ]:
from torchvision.utils import save_image
import cv2 as cv

def mels_to_png(mel_png):
    for image, label, audio_filename in mel_png:
        # dostaje 1 zjdecie i 1 label sprawdzic shape
        image = torch.squeeze(image)
        #print(image.shape)
        #print(torch.min(image))
        #print(label.shape)
        #print(label)
        #print(audio_filename) # nazwa pliku
        save_image(image, 'image.png')
        break
    return image
original_tensor = mels_to_png(mel_png)
print(torch.min(original_tensor))

def read_pnt_to_mel(png_path):
    image = cv.imread(png_path)
    image = torch.from_numpy(image)
    return image

loaded_tensor = read_pnt_to_mel("/content/image.png")

loaded_tensor = loaded_tensor.permute(2, 0, 1)
print(torch.min(loaded_tensor))

tensor(-35.1965)
tensor(0, dtype=torch.uint8)


# inaa

In [ ]:
audio_classes = ['blues', 'classical', 'electronic', 'electronic(vibe)', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
              '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
              '#bcbd22', '#17becf']
label_dict = {
    "blues" : 0,
    "classical" : 1,
    "electronic" : 2,
    "electronic(vibe)" : 3,
    "hiphop" : 4,
    "jazz" : 5,
    "metal" : 6,
    "pop" : 7,
    "reggae" : 8,
    "rock" : 9
}

In [ ]:
import pandas as pd
from torch.utils.data import random_split
import numpy as np

############## POEKSPERYMENTOWAC Z BATCH_SIZE
BATCH_SIZE = 16

df = pd.read_csv("/content/CNN_CLASIFIER_annotations_withgtzan_v2.csv")
data_path = "/content/dataset/"

myds = SoundDS(df, data_path, duration=4000, shift_pct=0.4)

# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, test_ds = random_split(myds, [num_train, num_val])

# Create training and validation data loaders
# batch_size jak dobierac spradzic
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

myds[0][0].shape

torch.Size([3, 64, 344])

# Baseline: Classification with softmax

1. RESNETS

In [ ]:


# Feature Extractor
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class EmbeddingNet(nn.Module):
    def __init__(self, latent_dim=128, architecture=18):
        super(EmbeddingNet, self).__init__()
        self.architecture = architecture
        self.feature_extractor = self._getResnet(latent_dim=latent_dim)

    def forward(self, x):
        x = self.feature_extractor(x)
        return x

    def _getResnet(self,latent_dim):
        '''
        Sprawdzic czy daje tez FC
        '''
        if self.architecture == 18:
            model = models.resnet18(pretrained=True)
        elif self.architecture == 34:
            model = models.resnet34(pretrained=True)
        elif self.architecture == 50:
            model = models.resnet50(pretrained=True)
        elif self.architecture == 101:
            model = models.resnet101(pretrained=True)
        elif self.architecture == 152:
            model = models.resnet152(pretrained=True)
        else: print("MODEL NOT LOADED")
        
        for param in model.parameters():
            param.requires_grad= False # freeze beggining layer
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, latent_dim)
    


        return model

    def get_embedding(self, x):
        return self.forward(x)

class ClassificationNet(nn.Module):
    def __init__(self, embedding_net, n_classes, latent_dim):
        super(ClassificationNet, self).__init__()
        self.embedding_net = embedding_net
        self.n_classes = n_classes
        self.latent_dim = latent_dim
        self.nonlinear = nn.PReLU()
        self.fc1 = nn.Linear(latent_dim, n_classes) # 

    def forward(self, x):
        output = self.embedding_net(x)
        output = self.nonlinear(output)
        scores = F.log_softmax(self.fc1(output), dim=-1)
        return scores

    def get_embedding(self, x):
        return self.nonlinear(self.embedding_net(x))

embed_model = EmbeddingNet(latent_dim=512, architecture=152)
x = torch.randn(1, 3, 64, 344)
output = embed_model(x)
print(output.shape)

classifi = ClassificationNet(embed_model, 10, latent_dim=512)
x = torch.randn(1, 3, 64, 344)
output_class = classifi(x)
print(output_class.shape)

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

torch.Size([1, 512])
torch.Size([1, 10])


In [ ]:


# Feature Extractor
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class EfficentEmbeddingNet(nn.Module):
    def __init__(self, latent_dim=128, architecture=0):
        super(EfficentEmbeddingNet, self).__init__()
        self.architecture = architecture
        self.feature_extractor = self._getEfficientNet(latent_dim=latent_dim)

    def forward(self, x):
        x = self.feature_extractor(x)
        return x

    def _getEfficientNet(self,latent_dim):
        '''
        Sprawdzic czy daje tez FC
        '''
        if self.architecture == 0:
            model = models.efficientnet_b0(pretrained=True)
        elif self.architecture == 1:
            model = models.efficientnet_b1(pretrained=True)
        elif self.architecture == 2:
            model = models.efficientnet_b2(pretrained=True)
        elif self.architecture == 3:
            model = models.efficientnet_b3(pretrained=True)
        elif self.architecture == 4:
            model = models.efficientnet_b4(pretrained=True)
        elif self.architecture == 5:
            model = models.efficientnet_b5(pretrained=True)
        elif self.architecture == 6:
            model = models.efficientnet_b6(pretrained=True)
        elif self.architecture == 7:
            model = models.efficientnet_b7(pretrained=True)
        else: print("MODEL NOT LOADED")
        
        # to czy napewno tak
        for param in model.parameters():
            param.requires_grad= False # freeze beggining layer
        model.classifier[1] = nn.Linear(in_features=1280, out_features=latent_dim)
        #print(model.classifier

        return model

    def get_embedding(self, x):
        return self.forward(x)

class EfficentClassificationNet(nn.Module):
    def __init__(self, embedding_net, n_classes, latent_dim):
        super(EfficentClassificationNet, self).__init__()
        self.embedding_net = embedding_net
        self.n_classes = n_classes
        self.latent_dim = latent_dim
        self.nonlinear = nn.PReLU()
        '''
        Both Flatten and GlobalAveragePooling2D are valid options. So is GlobalMaxPooling2D.

        Flatten will result in a larger Dense layer afterwards, which is 
        more expensive and may result in worse overfitting. But if you have lots of data, it might also perform better.

        As usual, it depends completely on your problem.
        '''
        self.fc1 = nn.Linear(latent_dim, n_classes) # 

    def forward(self, x):
        output = self.embedding_net(x)
        output = self.nonlinear(output)
        scores = F.log_softmax(self.fc1(output), dim=-1)
        return scores

    def get_embedding(self, x):
        return self.nonlinear(self.embedding_net(x))

embed_model = EfficentEmbeddingNet(latent_dim=512, architecture=0)
x = torch.randn(1, 3, 64, 344)
output = embed_model(x)
print(output.shape)

classifi = EfficentClassificationNet(embed_model, 10, latent_dim=512)
x = torch.randn(1, 3, 64, 344)
output_class = classifi(x)
print(output_class.shape)

torch.Size([1, 512])
torch.Size([1, 10])


In [ ]:


# Feature Extractor
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class EfficentEmbeddingNet_other(nn.Module):
    def __init__(self, latent_dim=128, architecture=0):
        super(EfficentEmbeddingNet, self).__init__()
        self.architecture = architecture
        self.feature_extractor = self._getEfficientNet(latent_dim=latent_dim)

    def forward(self, x):
        x = self.feature_extractor(x)
        return x

    def _getEfficientNet(self,latent_dim):
        '''
        Sprawdzic czy daje tez FC
        '''
        if self.architecture == 0:
            model = models.efficientnet_b0(pretrained=True)
        elif self.architecture == 1:
            model = models.efficientnet_b1(pretrained=True)
        elif self.architecture == 2:
            model = models.efficientnet_b2(pretrained=True)
        elif self.architecture == 3:
            model = models.efficientnet_b3(pretrained=True)
        elif self.architecture == 4:
            model = models.efficientnet_b4(pretrained=True)
        elif self.architecture == 5:
            model = models.efficientnet_b5(pretrained=True)
        elif self.architecture == 6:
            model = models.efficientnet_b6(pretrained=True)
        elif self.architecture == 7:
            model = models.efficientnet_b7(pretrained=True)
        else: print("MODEL NOT LOADED")
        
        # to czy napewno tak
        for param in model.parameters():
            param.requires_grad= False # freeze beggining layer 
        
        # przed 1280 na 10   >> global_average_pooling2d >> batch_normalization >>  (Dropout)  >> dense_6 (Dense) 1280 na 10
        model.classifier[1] = nn.Linear(in_features=1280, out_features=10)
        #print(model.classifier

        return model

    def get_embedding(self, x):
        return self.forward(x)

class EfficentClassificationNet(nn.Module):
    def __init__(self, embedding_net, n_classes, latent_dim):
        super(EfficentClassificationNet, self).__init__()
        self.embedding_net = embedding_net
        self.n_classes = n_classes
        self.latent_dim = latent_dim
        self.nonlinear = nn.PReLU()
        '''
        Both Flatten and GlobalAveragePooling2D are valid options. So is GlobalMaxPooling2D.

        Flatten will result in a larger Dense layer afterwards, which is 
        more expensive and may result in worse overfitting. But if you have lots of data, it might also perform better.

        As usual, it depends completely on your problem.
        '''
        self.fc1 = nn.Linear(latent_dim, n_classes) # 

    def forward(self, x):
        output = self.embedding_net(x)
        output = self.nonlinear(output)
        scores = F.log_softmax(self.fc1(output), dim=-1)
        return scores

    def get_embedding(self, x):
        return self.nonlinear(self.embedding_net(x))

embed_model = EfficentEmbeddingNet(latent_dim=512, architecture=0)
x = torch.randn(1, 3, 64, 344)
output = embed_model(x)
print(output.shape)

classifi = EfficentClassificationNet(embed_model, 10, latent_dim=512)
x = torch.randn(1, 3, 64, 344)
output_class = classifi(x)
print(output_class.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models


model = models.efficientnet_b0(pretrained=True)

for param in model.parameters():
    param.requires_grad= False


model
#model.classifier[1] = nn.Linear(in_features=1280, out_features=10)

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

In [ ]:
model.classifier[1] = nn.Linear(in_featyures, 10)

AdaptiveAvgPool2d(output_size=1)

In [ ]:
# idal train loop
'''
2. early stopping
3. easy connect to weights and biases
4. latwa w modulowaniu
5. loss/accuracy i inne jesli potrzebne
6. tqdm albo w&b
7. print co iles mini batch np 10% epoki
'''
def train_epoch(train_loader, optimizer, device, model, loss_fn, log_interval):
    running_loss = 0.
    train_loss = 0.
    for i, data in enumerate(train_loader):
        images, labels = data
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        preds = model(images)
        loss = loss_fn(preds, labels)
        loss.backward() # Calculates the backward gradients over the learning weights

        optimizer.step() # Tells the optimizer to perform one learning step - that is, adjust the model’s learning weights
                         # based on the observed gradients for this batch, according to the optimization algorithm we chose It
                         #  reports on the loss for every log_interval. (log_interval = 500) -> every 500 batches
        running_loss += loss.item()
        if i % log_interval == 0:
            train_loss = running_loss / loss # loss per batch
            print('  batch {} loss: {}'.format(i + 1, train_loss))
            running_loss = 0.
    return train_loss # loss calej epoki

def test_epoch(test_loader, model, device, loss_fn, save_best=False):
    '''
    OUTPUT
    jesli chce sie dokladnosc w procentach to pomnozyc correct * 100 !
    '''
    best_accuracy = 0
    test_loss = 0
    correct = 0
    size = len(test_loader.dataset) # ? czy to naprwno tak sprawdzic
    num_batches = len(test_loader)  # ?
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            preds = model(images)
            test_loss += loss_fn(preds, labels).item()
            correct += (preds.argmax(1) == labels).type(torch.float).sum().item()
    test_loss = test_loss / num_batches
    correct = correct / size
    if save_best is True:
        if correct > best_accuracy:
            torch.save(model, 'best-model.pt')
            torch.save(model.state_dict(), 'best-model-parameters.pt')
            best_accuracy = correct
    print("Accuracy {}, Test_loss {}".format(correct*100, test_loss))
    return correct, test_loss

def train_model(epoch, model, train_loader, test_loader, optimizer, device, loss_fn, scheduler, log_interval, save_best=False, start_epoch=0):
    for i in range(0, start_epoch): # po co ? bo dziala od ktorejs epoki czasami !!!
        scheduler.step()

    train_loss_tab = []
    test_loss_tab = []
    accuracy_tab = []
    for i in range(start_epoch, epoch):
        train_loss = train_epoch(train_loader, optimizer, device, model, loss_fn, log_interval)
        accuracy, test_loss = test_epoch(test_loader, model, device, loss_fn, save_best=False)
        scheduler.step()
        train_loss_tab.append(train_loss)
        test_loss_tab.append(test_loss)
        accuracy_tab.append(accuracy)
    return train_loss_tab, test_loss_tab, accuracy_tab


In [ ]:
import torch.optim.lr_scheduler
import torch.optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

latent_dim = 128

embedding_model = EmbeddingNet(latent_dim=latent_dim, architecture=34).to(device)
model_classification = ClassificationNet(embedding_model, 10, latent_dim=latent_dim).to(device)

loss_fn = torch.nn.NLLLoss()
lr = 1e-2
optimizer = torch.optim.Adam(model_classification.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 5
log_interval = 50 # ?


cuda:0


In [ ]:
train_tab, test_tab, acc_tab = train_model(epoch=n_epochs,
                                           model=model_classification,
                                           train_loader=train_dl,
                                           test_loader=test_dl,
                                           optimizer=optimizer,
                                           device=device,
                                           loss_fn=loss_fn,
                                           scheduler=scheduler,
                                           log_interval=100, # to lepiej ogarnac
                                           save_best=True,
                                           start_epoch=0)
# 25min 1 epoka na resnet  50  - 30 sek dane
# 24:47 start 15min  epoka jakos
# preprocess danych nie jest nma gpu chyba dlatego tak dlugo to trwa

  batch 1 loss: 1.0
  batch 101 loss: 100.00543975830078
  batch 201 loss: 102.75122833251953
  batch 301 loss: 105.60599517822266
Accuracy 36.0919540229885, Test_loss 1.7908680148240996
  batch 1 loss: 1.0
  batch 101 loss: 92.45731353759766
  batch 201 loss: 96.13599395751953
  batch 301 loss: 121.52803039550781
Accuracy 33.639846743295024, Test_loss 1.8284321732637359
  batch 1 loss: 1.0
  batch 101 loss: 96.44847869873047
  batch 201 loss: 111.07157135009766
  batch 301 loss: 85.91714477539062
Accuracy 37.01149425287356, Test_loss 1.8034587139036597
  batch 1 loss: 1.0
  batch 101 loss: 108.36941528320312
  batch 201 loss: 123.77244567871094
  batch 301 loss: 81.65238189697266
Accuracy 38.31417624521073, Test_loss 1.6580796619740927
  batch 1 loss: 1.0
  batch 101 loss: 93.16629028320312
  batch 201 loss: 111.26626586914062
  batch 301 loss: 114.9343032836914
Accuracy 40.689655172413794, Test_loss 1.671766534084227


In [ ]:
# rozpisac ktore zle/dobrze klasyfikuje z kazdego gatunku i dlaczwego 
# ten wykres jebany se zrobic
# Dataset usprawnic

In [ ]:
def extract_embeddings(dataloader, model):
    with torch.no_grad():
        model.eval()
        embeddings = np.zeros((len(dataloader.dataset), 2))
        labels = np.zeros(len(dataloader.dataset))
        k = 0
        for images, target in dataloader:
            if cuda:
                images = images.cuda()
            embeddings[k:k+len(images)] = model.get_embedding(images).data.cpu().numpy()
            labels[k:k+len(images)] = target.numpy()
            k += len(images)
    return embeddings, labels

In [ ]:
#Reduce with PCA or Tsne 128 dim to only 2
# pca wszytsjkie naraz
# w ktorych mozna dorzucac jak w embedding; na apke

In [ ]:
def plot_embeddings(embeddings, targets, xlim=None, ylim=None):
    plt.figure(figsize=(10,10))
    for i in range(10):
        inds = np.where(targets==i)[0]
        plt.scatter(embeddings[inds,0], embeddings[inds,1], alpha=0.5, color=colors[i])
    if xlim:
        plt.xlim(xlim[0], xlim[1])
    if ylim:
        plt.ylim(ylim[0], ylim[1])
    plt.legend(mnist_classes)

In [ ]:
# Plot embeddings in Weights and biases

In [ ]:
pip install torchinfo

In [ ]:
from torchinfo import summary
summary(model_classification, input_size=(1, 3, 64, 2579))

Layer (type:depth-idx)                             Output Shape              Param #
ClassificationNet                                  --                        --
├─EmbeddingNet: 1-1                                [1, 512]                  --
│    └─ResNet: 2-1                                 [1, 512]                  --
│    │    └─Conv2d: 3-1                            [1, 64, 32, 1290]         (9,408)
│    │    └─BatchNorm2d: 3-2                       [1, 64, 32, 1290]         (128)
│    │    └─ReLU: 3-3                              [1, 64, 32, 1290]         --
│    │    └─MaxPool2d: 3-4                         [1, 64, 16, 645]          --
│    │    └─Sequential: 3-5                        [1, 256, 16, 645]         (215,808)
│    │    └─Sequential: 3-6                        [1, 512, 8, 323]          (2,339,840)
│    │    └─Sequential: 3-7                        [1, 1024, 4, 162]         (40,613,888)
│    │    └─Sequential: 3-8                        [1, 2048, 2, 81]          (14,

# TODO

1. preprocess szybszy (zmiana na zdjecia gotowe juz)
2. W&B
3. model z gtzan < -- (do tego w&b ny sie przydalo)

# W&B Experiments

In [ ]:
import pandas as pd
from torch.utils.data import random_split
import numpy as np

In [ ]:
%%capture
!pip install wandb --upgrade

In [ ]:
import wandb

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
sweep_config = {
    'method' : 'random',
}
metric = {
    'name' : 'test_loss',
    'goal' : 'minimize'
}
sweep_config['metric'] = metric
# name the hperparameters
parameters_dict = {
    'epochs': {
        'values': [3, 5]
    },
    'batch_size': {
        'values': [16, 32, 64]
    },
    'duration': {
        'values': [4000, 8000, 15000, 25000] # /1000 zeby bylo w sekundach
    },
    'shift_pct': {
        'values': [0.4, 0.2]
    },
    'latent_dim': {
        'values': [256, 512]
    },
    'optimizer': {
        'values': ['adam', 'sgd', 'RMSprop'] # z scheduler
    },
    'scheduler': {
        'values': ['ExponentialLR', 'CyclicLR - exp_range'] # z scheduler
    },
    'models': {
        'values': ['Resnet34', 'EfficentNet0'] # z scheduler
    }
}
sweep_config['parameters'] = parameters_dict

In [ ]:
from pprint import pprint
pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'test_loss'},
 'parameters': {'batch_size': {'values': [16, 32, 64]},
                'duration': {'values': [4000, 8000, 15000, 25000]},
                'epochs': {'values': [3, 5]},
                'latent_dim': {'values': [256, 512]},
                'models': {'values': ['Resnet34', 'EfficentNet0']},
                'optimizer': {'values': ['adam', 'sgd', 'RMSprop']},
                'scheduler': {'values': ['ExponentialLR',
                                         'CyclicLR - exp_range']},
                'shift_pct': {'values': [0.4, 0.2]}}}


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="MUSIC_CLASSIFICATION")

Create sweep with ID: xpt0sfaz
Sweep URL: https://wandb.ai/wualas/MUSIC_CLASSIFICATION/sweeps/xpt0sfaz


In [ ]:
def build_network(model_name, latent_dim):
    # do zmiany ale izi
    if model_name == 'Resnet18':
        architecture = 18
        embed_model = EmbeddingNet(latent_dim=latent_dim, architecture=architecture).to(device)
        model = ClassificationNet(embed_model, 10, latent_dim=latent_dim).to(device)
    if model_name == 'Resnet34':
        architecture = 34
        embed_model = EmbeddingNet(latent_dim=latent_dim, architecture=architecture).to(device)
        model = ClassificationNet(embed_model, 10, latent_dim=latent_dim).to(device)
    if model_name == 'EfficentNet0':
        architecture = 0
        embed_model = EfficentEmbeddingNet(latent_dim=latent_dim, architecture=architecture).to(device)
        model = EfficentClassificationNet(embed_model, 10, latent_dim=latent_dim).to(device)
    if model_name == 'MyFuckingNetwork':
        model = MyFuckingNetwork().to(device)
    return model

def build_optimizer(network, optimizer):
    if optimizer == "sgd":
        optimizer = torch.optim.SGD(network.parameters(),
                              lr=0.01, momentum=0.9)
    elif optimizer == "adam":
        optimizer = torch.optim.Adam(network.parameters(),
                               lr=3e-4)
    elif optimizer == 'RMSprop':
        optimizer = torch.optim.RMSprop(network.parameters(),
                                lr=0.01)
    return optimizer

def build_scheduler(optimizer, scheduler):
    '''
    w schedulerach jest tyle hiperparametrow ze japierodle
    '''
    if scheduler == 'ExponentialLR':
      scheduler = torch.optim.lr_scheduler.ExponentialLR(
          optimizer, gamma=0.1
      )
    elif scheduler == 'CyclicLR - exp_range':
      scheduler = torch.optim.lr_scheduler.CyclicLR(
          optimizer, base_lr=0.001, max_lr=0.1,step_size_up=5,
          mode="exp_range",gamma=0.85, cycle_momentum=False
      )
    return scheduler

def build_loader(df, data_path, batch_size, duration=30000, shift_pct=0.4, experimental=None):
    '''
    experimental = 0.6 menas that 60% of dataset will be deleted for wandb research
    '''
    myds = SoundDS(df, data_path, duration=duration, shift_pct=shift_pct)

    # Random split of 80:20 between training and validation
    num_items = len(myds)


    if experimental != None:
        to_remove = round(num_items * experimental)
        num_items = len(myds) - to_remove
        num_train = round(num_items * 0.8)
        num_val = num_items - num_train
        remove_ds, train_ds, test_ds = random_split(myds, [to_remove,num_train, num_val])
        print("WORKS")
    else:
        num_train = round(num_items * 0.8)
        num_val = num_items - num_train
        train_ds, test_ds = random_split(myds, [num_train, num_val])
    # Create training and validation data loaders
    # batch_size jak dobierac spradzic
    train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    test_dl = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)

    return train_dl, test_dl

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_fn = nn.CrossEntropyLoss()
df = pd.read_csv("/content/CNN_CLASIFIER_annotations_withgtzan_v2.csv")
data_path = "/content/dataset/"

def train_model_wandb(config=None):
    with wandb.init(config=config):
        config = wandb.config
        train_dl, test_dl = build_loader(df, data_path, config.batch_size, config.duration, config.shift_pct, experimental=0.6)
        model = build_network(config.models, config.latent_dim)
        optimizer = build_optimizer(model, config.optimizer)
        scheduler = build_scheduler(optimizer, config.scheduler)
        for epoch in range(config.epochs):
            print(f"Epoch {epoch+1}\n-------------------------------")
            train_loss = train_epoch(train_dl, optimizer, device, model, loss_fn, 100)
            accuracy, test_loss = test_epoch(test_dl, model, device, loss_fn, save_best=False)
            scheduler.step()
            wandb.log({'epoch': epoch, 'test_loss': test_loss, 'train_loss': train_loss, 'accuracy': accuracy})

In [ ]:
wandb.agent(sweep_id, function=train_model_wandb, count=5)

wandb: Agent Starting Run: xm369377 with config:
wandb: 	batch_size: 8
wandb: 	duration: 8000
wandb: 	epochs: 4
wandb: 	latent_dim: 512
wandb: 	models: EfficentNet0
wandb: 	optimizer: sgd
wandb: 	scheduler: ExponentialLR
wandb: 	shift_pct: 0.2


WORKS
Epoch 1
-------------------------------
  batch 1 loss: 1.0
  batch 101 loss: 113.75459289550781
  batch 201 loss: 104.04815673828125
Accuracy 39.272030651341, Test_loss 1.6987473495078809
Epoch 2
-------------------------------
  batch 1 loss: 1.0
  batch 101 loss: 118.47662353515625
  batch 201 loss: 76.14639282226562
Accuracy 47.509578544061306, Test_loss 1.544988650264162
Epoch 3
-------------------------------
  batch 1 loss: 1.0
  batch 101 loss: 121.53935241699219
  batch 201 loss: 130.8271484375
Accuracy 46.36015325670498, Test_loss 1.5702572853276224
Epoch 4
-------------------------------
  batch 1 loss: 0.9999999403953552
  batch 101 loss: 96.98963165283203
  batch 201 loss: 127.63470458984375
Accuracy 44.827586206896555, Test_loss 1.5866799769979534



accuracy,▁█▇▆
epoch,▁▃▆█
test_loss,█▁▂▃
train_loss,▅▁██
accuracy,0.44828
epoch,3
test_loss,1.58668
train_loss,127.6347


wandb: Agent Starting Run: cpn47oy6 with config:
wandb: 	batch_size: 16
wandb: 	duration: 4000
wandb: 	epochs: 6
wandb: 	latent_dim: 512
wandb: 	models: Resnet34
wandb: 	optimizer: RMSprop
wandb: 	scheduler: CyclicLR - exp_range
wandb: 	shift_pct: 0.4


WORKS
Epoch 1
-------------------------------
  batch 1 loss: 1.0
  batch 101 loss: 141.89784240722656
Accuracy 32.18390804597701, Test_loss 1.8987957239151
Epoch 2
-------------------------------
  batch 1 loss: 1.0
  batch 101 loss: 3644.871337890625
Accuracy 19.157088122605366, Test_loss 2.2052187197136157
Epoch 3
-------------------------------
  batch 1 loss: 1.0
  batch 101 loss: 3664.314697265625
Accuracy 21.455938697318008, Test_loss 2.222345297986811
Epoch 4
-------------------------------
  batch 1 loss: 1.0
  batch 101 loss: 12126.6025390625
Accuracy 28.35249042145594, Test_loss 2.1966320420756484
Epoch 5
-------------------------------
  batch 1 loss: 1.0
  batch 101 loss: 2.0980050563812256
Accuracy 20.689655172413794, Test_loss 22.389896884109035
Epoch 6
-------------------------------
  batch 1 loss: 1.0
  batch 101 loss: 163.32737731933594
Accuracy 6.321839080459771, Test_loss 23.241143255522758



...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: qwgvj81i with config:
wandb: 	batch_size: 32
wandb: 	duration: 4000
wandb: 	epochs: 2
wandb: 	latent_dim: 256
wandb: 	models: Resnet18
wandb: 	optimizer: adam
wandb: 	scheduler: CyclicLR - exp_range
wandb: 	shift_pct: 0.6


WORKS
Epoch 1
-------------------------------
  batch 1 loss: 1.0
Accuracy 36.59003831417624, Test_loss 1.8100581519743975
Epoch 2
-------------------------------
  batch 1 loss: 1.0
Accuracy 31.03448275862069, Test_loss 2.0656288511612835



accuracy,█▁
epoch,▁█
test_loss,▁█
train_loss,▁▁
accuracy,0.31034
epoch,1
test_loss,2.06563
train_loss,1.0


wandb: Agent Starting Run: vcxjrsc4 with config:
wandb: 	batch_size: 64
wandb: 	duration: 25000
wandb: 	epochs: 2
wandb: 	latent_dim: 512
wandb: 	models: Resnet18
wandb: 	optimizer: sgd
wandb: 	scheduler: CyclicLR - exp_range
wandb: 	shift_pct: 0.2


WORKS
Epoch 1
-------------------------------
  batch 1 loss: 1.0
Accuracy 24.71264367816092, Test_loss 2.1506599850124783
Epoch 2
-------------------------------
  batch 1 loss: 1.0
